In [ ]:
"""
Created on Wed Oct 11 11:59 2023

Explore the effect of the ensemble of NNs

Author: @claraburgard

"""

In [ ]:
import xarray as xr
import matplotlib as mpl
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
%matplotlib qt5

In [ ]:
mod_size =  'small' #'mini', 'small', 'medium', 'large', 'extra_large'
TS_opt = 'extrap' # extrap, whole, thermocline
norm_method =  'std' # std, interquart, minmax
exp_name = 'newbasic2'#'allbutconstants' #'onlyTSdraftandslope' #'TSdraftbotandiceddandwcd' #'onlyTSisfdraft' #'TSdraftbotandiceddandwcdreldGL' #TSdraftslopereldGL

In [ ]:
nemo_run = 'bf663'#'bi646' #bf663

In [ ]:
outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/SMITH_'+nemo_run+'/'
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'


In [ ]:
res_2D_list = []
for tt in range(1980,1980+60):
    res_2D = xr.open_dataset(outputpath_melt_nn + 'evalmetrics_2D_'+mod_size+'_'+exp_name+'_ensstd_'+TS_opt+'_norm'+norm_method+'_'+str(tt)+'_'+nemo_run+'.nc')
    res_2D_list.append(res_2D.assign_coords({'time': tt}))
res_2D_allyy = xr.concat(res_2D_list, dim='time')

In [ ]:
res_2D_allyy['predicted_melt'].mean('time').plot(vmax=20,cmap=mpl.cm.Reds)

In [ ]:
res_2D_allyy['predicted_melt'].mean('time').mean()

In [ ]:
res_1D_list = []
target_1D_list = []

for tt in range(1980,1980+60):
    res_1D = xr.open_dataset(outputpath_melt_nn + 'evalmetrics_1D_'+mod_size+'_'+exp_name+'_allmembers_'+TS_opt+'_norm'+norm_method+'_'+str(tt)+'_'+nemo_run+'.nc')
    res_1D_list.append(res_1D)
    
    NEMO_melt_rates_1D = xr.open_dataset(inputpath_data+'melt_rates_1D_NEMO_oneFRIS_'+str(tt)+'.nc')
    target_melt_Gt_yr = NEMO_melt_rates_1D['melt_Gt_per_y_tot']
    target_1D_list.append(target_melt_Gt_yr)
    
res_1D_allyy = xr.concat(res_1D_list, dim='time')    
target_1D_allyy = xr.concat(target_1D_list, dim='time')

In [ ]:
# File isf
inputpath_mask='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_1980.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','East 1','East 2','Dronning Maud Land']

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

#sorted_isf_all = [11,69,43,12,70,44,29,13,58,71,45,30,31,61,73,47,32,48,33,17,49,34,18,10,65,51,22,38,52,23,66,53,39,24,40,54,75,25,26,42,55] #62

for kisf in tqdm(file_isf.Nisf.drop_sel(Nisf=4).values):
    
        
    ax[i] = f.add_subplot(7,7,i+1)
    
    for sn in res_1D_allyy.seed_nb:
        
        ax[i].plot(res_1D_allyy.time, res_1D_allyy['predicted_melt'].sel(seed_nb=sn, Nisf=kisf,metrics='Gt'), color='k', alpha=0.1)
    
    ax[i].plot(res_1D_allyy.time, res_1D_allyy['predicted_melt'].mean('seed_nb').sel(Nisf=kisf,metrics='Gt'), color='k', linewidth=2)

    #if kisf == 23:
    #    ax[i].set_title('Tracy Tremenchus')
    #elif kisf == 24:
    #    ax[i].set_title('Conger/Glenzer')
    #elif kisf == 110:
    #    ax[i].set_title('Ekström')
    #else:
    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(0,60)
    #ax[i].axvline(x=30, c='k', linestyle='--')

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
#plt.savefig(plot_path+'timeseries_ensembles_smallNN.pdf')

In [ ]:
diff_ens = res_1D_allyy['predicted_melt'].sel(metrics='Gt') - target_1D_allyy
diff_ensmean = res_1D_allyy['predicted_melt'].sel(metrics='Gt').mean('seed_nb') - target_1D_allyy

In [ ]:
rmse = np.sqrt((diff_ens**2).mean(['Nisf','time']))
rmse_ensmean = np.sqrt((diff_ensmean**2).mean(['Nisf','time']))

In [ ]:
rmse.max()

In [ ]:
rmse.min()

In [ ]:
rmse_ensmean

In [ ]:
res_1D_allyy['predicted_melt'].sel(seed_nb=sn, Nisf=kisf)#,metrics='Gt')

In [ ]:
for seed_nb
res_1D_allyy['predicted_melt'].sel(metrics='Gt',Nisf=66)

In [ ]:
res_1D_test1980['predicted_melt'].isel(time=0).isel(seed_nb=9).load()

In [ ]:
res_1D_allyy['predicted_melt'].sel(seed_nb=1,metrics='Gt').isel(time=range(10))